In [24]:
import pandas as pd

data = pd.read_csv(r'C:\Users\satv\Downloads\Bank_Personal_Loan_Modelling.csv')


In [25]:
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split


X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Convert categorical variables to numerical variables
X = pd.get_dummies(X, columns=['Education', 'Family', 'CCAvg', 'Securities Account', 'CD Account', 'Online'])

# Check column names again
print(X.columns)

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3, random_state=0)

# Reshape arrays
x_train = x_train.reshape(-1, x_train.shape[1])
y_train = y_train.reshape(-1, 1)

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Mortgage',
       'Personal Loan', 'Education_1', 'Education_2', 'Education_3',
       ...
       'CCAvg_8.9', 'CCAvg_9.0', 'CCAvg_9.3', 'CCAvg_10.0',
       'Securities Account_0', 'Securities Account_1', 'CD Account_0',
       'CD Account_1', 'Online_0', 'Online_1'],
      dtype='object', length=128)


In [27]:
import numpy as np
import random

class PSO_NN:
    def __init__(self, n_input, n_hidden, n_output, swarm_size=50, max_iterations=1000, c1=2.0, c2=2.0, w=0.7):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.n_output = n_output
        self.swarm_size = swarm_size
        self.max_iterations = max_iterations
        self.c1 = c1
        self.c2 = c2
        self.w = w
        self.swarm_pos = np.zeros((swarm_size, (n_input * n_hidden) + n_hidden + (n_hidden * n_output) + n_output))
        self.swarm_vel = np.zeros((swarm_size, (n_input * n_hidden) + n_hidden + (n_hidden * n_output) + n_output))
        self.personal_best_pos = np.zeros((swarm_size, (n_input * n_hidden) + n_hidden + (n_hidden * n_output) + n_output))
        self.personal_best_val = np.full(swarm_size, float('inf'))
        self.global_best_pos = np.zeros((n_input * n_hidden) + n_hidden + (n_hidden * n_output) + n_output)
        self.global_best_val = float('inf')

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, x, theta):
        hidden_weights = np.reshape(theta[:self.n_input * self.n_hidden], (self.n_input, self.n_hidden))
        hidden_bias = np.reshape(theta[self.n_input * self.n_hidden:self.n_input * self.n_hidden + self.n_hidden], self.n_hidden)
        output_weights = np.reshape(theta[self.n_input * self.n_hidden + self.n_hidden:self.n_input * self.n_hidden + self.n_hidden + self.n_hidden * self.n_output], (self.n_hidden, self.n_output))
        output_bias = np.reshape(theta[self.n_input * self.n_hidden + self.n_hidden + self.n_hidden * self.n_output:], self.n_output)
        hidden_activation = self.sigmoid(np.dot(x, hidden_weights) + hidden_bias)
        output_activation = np.dot(hidden_activation, output_weights) + output_bias
        return output_activation

    def fitness(self, x, y, theta):
        output_activation = self.forward(x, theta)
        return np.sum(np.square(y - output_activation))
    
    def train(self, x_train, y_train):
        for i in range(self.swarm_size):
            self.swarm_pos[i] = np.random.uniform(-1, 1, size=(self.n_input * self.n_hidden) + self.n_hidden + (self.n_hidden * self.n_output) + self.n_output)
            fitness = self.fitness(x_train, y_train, self.swarm_pos[i])
            self.personal_best_pos[i] = self.swarm_pos[i]
            self.personal_best_val[i] = fitness
            if fitness < self.global_best_val:
                self.global_best_val = fitness
                self.global_best_pos = self.swarm_pos[i]

        for iteration in range(self.max_iterations):
            for i in range(self.swarm_size):
                self.swarm_vel[i] = self.w * self.swarm_vel[i] + self.c1 * random.uniform(0, 1) * (self.personal_best_pos[i] - self.swarm_pos[i]) + self.c2 * random.uniform(0, 1) * (self.global_best_pos - self.swarm_pos[i])
                self.swarm_pos[i] += self.swarm_vel[i]
                fitness = self.fitness(x_train, y_train, self.swarm_pos[i])
                if fitness < self.personal_best_val[i]:
                    self.personal_best_val[i] = fitness
                    self.personal_best_pos[i] = self.swarm_pos[i]
                    if fitness < self.global_best_val:
                        self.global_best_val = fitness
                        self.global_best_pos = self.swarm_pos[i]

    def predict(self, x):
        output_activation = self.forward(x, self.global_best_pos)
        return np.round(output_activation)


In [28]:
nn = PSO_NN(n_input=x_train.shape[1], n_hidden=10, n_output=1)
nn.train(x_train, y_train)
y_pred = nn.predict(x_test)

accuracy = (y_test == y_pred.reshape(-1)).mean()
print(f"Accuracy: {accuracy}")

C:\Users\saich\AppData\Local\Temp\ipykernel_472\2698907520.py:22: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Accuracy: 0.7033333333333334
